In [ ]:
import numpy as np
import tifffile as tiff
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim
from utils import log

def load_volume(path):
    return tiff.imread(path).astype(np.float32)

def normalize(volume):
    return volume / 65535.0  # normalize 16-bit to [0, 1]

def compare_volumes(gt, method, mask, method_name):
    """Evaluate PSNR, SSIM, MAE, and MeanIntensityError on masked slices only."""
    psnr_list = []
    ssim_list = []
    l1_list = []

    D = gt.shape[0]
    num_masked = 0

    for i in range(D):
        if mask[i].mean() > 0.5:  # Only evaluate corrupted slices
            gt_slice = gt[i]
            pred_slice = method[i]

            l1 = np.mean(np.abs(gt_slice - pred_slice))
            mse = np.mean((gt_slice - pred_slice) ** 2)

            if mse == 0:
                psnr_score = 100.0
            else:
                psnr_score = psnr(gt_slice, pred_slice, data_range=1.0)

            ssim_score = ssim(gt_slice, pred_slice, data_range=1.0)

            l1_list.append(l1)
            psnr_list.append(psnr_score)
            ssim_list.append(ssim_score)
            num_masked += 1

    if num_masked == 0:
        log(f"{method_name}:")
        log("  No masked slices found to evaluate.")
        log("-" * 40)
        return None

    avg_psnr = np.mean(psnr_list)
    avg_ssim = np.mean(ssim_list)
    avg_l1 = np.mean(l1_list)
    intensity_diff = np.abs(gt.mean() - method.mean())

    log(f"{method_name}:")
    log(f"  Mean PSNR (masked slices): {avg_psnr:.2f} dB")
    log(f"  Mean SSIM (masked slices): {avg_ssim:.4f}")
    log(f"  Mean L1   (masked slices): {avg_l1:.6f}")
    log(f"  Mean Intensity Error     : {intensity_diff:.6f}")
    log("-" * 40)

def run_comparison(gt_path, predicted_path_1, predicted_path_2, mask_path):
    # Load volumes
    gt = normalize(load_volume(gt_path))
    pred1 = normalize(load_volume(predicted_path_1))
    pred2 = normalize(load_volume(predicted_path_2))
    mask = tiff.imread(mask_path)  # mask expected as uint8 or uint16 0/1 format

    # Ensure shapes match
    assert gt.shape == pred1.shape == pred2.shape == mask.shape, "Volumes and mask must match in shape!"

    # Compare volumes
    compare_volumes(gt, pred1, mask, "Predicted Volume 1")
    compare_volumes(gt, pred2, mask, "Predicted Volume 2")
    print('\n')


In [ ]:
base_path = "/media/admin/Expansion/Mosaic_Data_for_Ipeks_Group/OCT_Inpainting_Testing/"

run_comparison(
    gt_path=f"{base_path}1.1_OCTA_Vol1_Processed_Cropped_gt.tif",
    predicted_path_1=f"{base_path}1.1_OCTA_Vol1_Processed_Cropped_inpainted_2p5DUNet_fold4_v2.tif",
    predicted_path_2=f"{base_path}1.1_OCTA_Vol1_Processed_Cropped_inpainted_2p5DUNet_fold4_v4.tif",
    mask_path=f"{base_path}1.1_OCTA_Vol1_Processed_Cropped_mask.tif"
)

run_comparison(
    gt_path=f"{base_path}1.2_OCTA_Vol2_Processed_Cropped_gt.tif",
    predicted_path_1=f"{base_path}1.2_OCTA_Vol2_Processed_Cropped_inpainted_2p5DUNet_fold1_v2.tif",
    predicted_path_2=f"{base_path}1.2_OCTA_Vol2_Processed_Cropped_inpainted_2p5DUNet_fold1_v4.tif",
    mask_path=f"{base_path}1.2_OCTA_Vol2_Processed_Cropped_mask.tif"
)

run_comparison(
    gt_path=f"{base_path}1.4_OCTA_Vol1_Processed_Cropped_gt.tif",
    predicted_path_1=f"{base_path}1.4_OCTA_Vol1_Processed_Cropped_inpainted_2p5DUNet_fold3_v2.tif",
    predicted_path_2=f"{base_path}1.4_OCTA_Vol1_Processed_Cropped_inpainted_2p5DUNet_fold3_v4.tif",
    mask_path=f"{base_path}1.4_OCTA_Vol1_Processed_Cropped_mask.tif"
)

# run_comparison(
#     gt_path=f"{base_path}3.4_OCT_uint16_Cropped_Reflected_VolumeSplit_2_RegSeq_seqSVD_gt.tif",
#     predicted_path_1=f"{base_path}3.4_OCT_uint16_Cropped_Reflected_VolumeSplit_2_RegSeq_seqSVD_inpainted_2p5DUNet_fold5_v2.tif",
#     predicted_path_2=f"{base_path}3.4_OCT_uint16_Cropped_Reflected_VolumeSplit_2_RegSeq_seqSVD_inpainted_2p5DUNet_fold5_v4.tif",
#     mask_path=f"{base_path}3.4_OCT_uint16_Cropped_Reflected_VolumeSplit_2_RegSeq_seqSVD_mask.tif"
# )

# run_comparison(
#     gt_path=f"{base_path}5.3_OCT_uint16_Cropped_Reflected_VolumeSplit_1_RegSeq_seqSVD_gt.tif",
#     predicted_path_1=f"{base_path}5.3_OCT_uint16_Cropped_Reflected_VolumeSplit_1_RegSeq_seqSVD_inpainted_2p5DUNet_fold2_v2.tif",
#     predicted_path_2=f"{base_path}5.3_OCT_uint16_Cropped_Reflected_VolumeSplit_1_RegSeq_seqSVD_inpainted_2p5DUNet_fold2_v4.tif",
#     mask_path=f"{base_path}5.3_OCT_uint16_Cropped_Reflected_VolumeSplit_1_RegSeq_seqSVD_mask.tif"
# )

Predicted Volume 1:
  Mean PSNR (missing slices only): 35.20 dB
  Mean SSIM (missing slices only): 0.8231
  Mean MAE  (missing slices only): 0.011292
  Mean MSE  (missing slices only): 0.000306
----------------------------------------
Predicted Volume 2:
  Mean PSNR (missing slices only): 34.97 dB
  Mean SSIM (missing slices only): 0.8232
  Mean MAE  (missing slices only): 0.011826
  Mean MSE  (missing slices only): 0.000324
----------------------------------------


Predicted Volume 1:
  Mean PSNR (missing slices only): 34.96 dB
  Mean SSIM (missing slices only): 0.8264
  Mean MAE  (missing slices only): 0.011655
  Mean MSE  (missing slices only): 0.000336
----------------------------------------
Predicted Volume 2:
  Mean PSNR (missing slices only): 35.66 dB
  Mean SSIM (missing slices only): 0.8292
  Mean MAE  (missing slices only): 0.011127
  Mean MSE  (missing slices only): 0.000275
----------------------------------------


Predicted Volume 1:
  Mean PSNR (missing slices only): 3